In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from scipy.integrate import odeint

In [9]:
print np.version.version
import scipy
print scipy.version.version

1.11.1
1.1.0
